# Desagregação dos dados da bacia do Parnaíba usando GeoPandas

Este *notebook* mostra é uma continuação [deste *notebook*](https://github.com/francisconog/PBCE/blob/master/analysis/ESTPLU_POR_BACIA.ipynb) que classifica um ponto de acordo com sua latitude e longitude. Em 2012 a Bacia do Parnaíba foi dividida em: Serra de Ibiapaba e Sertões de Crateús. Desta forma, precisa-se desagregar os dados da antiga bacia para a realização de análises mais completas.

*Este Notebook faz parte de uma colaboração da Universidade Federal do Ceará (UFC), da Companhia de Gestão de Recursos Hídricos (COGERH) e da Fundação Cearense de Apoio ao Desenvolvimento Científico e Tecnológico (FUNCAP) no âmbito do projeto de desenvolvimento de ferramentas tecnológicas de gestão para planejamento de recursos hídricos do estado do Ceará: Segurança hídrica e planejamento de secas*

*Code by: Francisco Nogueira*

## Importing packages

In [1]:
import geobr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from shapely.geometry import Point
from tqdm import tqdm
import sys
import warnings
sys.path.append('../src')
warnings.filterwarnings('ignore')
from functions import *

## Paths

In [2]:
data_path = "../data/"
shape_file_path = "shapes"
shape_malha_dagua = "Drenagem/Margens_simples_CE_corrigidoSRH.shp"
shape_bacias = "geomap_bacias/i3geomap_bacias.shp"
outorga_data_path = "Dados Alocar/outorga-cogerh"

## Import Data

In [3]:
# Shape do Estado do Ceará
state = geobr.read_state(code_state="CE", year=2018)


In [4]:
# Shape das malhas d'água do estado do Ceará
shapefile_malha = gpd.read_file(os.path.join(data_path,shape_file_path,shape_malha_dagua)).to_crs("EPSG:31984")

# Shape das bacias hidrográfica do Ceará
shapefile_bacias = gpd.read_file(os.path.join(data_path,shape_file_path,shape_bacias),
                                encoding='utf-8').to_crs("EPSG:31984")


In [5]:
# Dados de outorga de 2012
outorga_2012 = pd.read_excel(os.path.join(data_path,outorga_data_path,"outorga_2012.xls"))
outorga_2012.head()


,Requerente,N Processo,Bacia,Tipo de Uso,Vz Outorgada(L/s),Vol. Outorgado(M3/ano),Latitude,Longitude,Manancial,Data Inicial,Data Final,Unnamed: 11,Unnamed: 12
0,ASS. DOS MORADORES DE QUIXABA DOS PAULOS E ADJ.,04279973-2,BAIXO JAGUARIBE,ABASTECIMENTO HUMANO,0.66,10300.00,NaN,NaN,CANAL DO TRABALHADOR,08/09/2004,08/09/2014,NaN,NaN
1,SAAE DE PEDRA BRANCA,03272770-4,BANABUIÚ,ABASTECIMENTO HUMANO,22.57,711750.00,NaN,NaN,AÇUDE TRAPIA II,30/08/2003,30/08/2013,NaN,NaN
2,ASS. DOS PRODUTORES RURAIS DE SABONETE,02413120-2,BANABUIÚ,ABASTECIMENTO HUMANO,2.22,35040.00,NaN,NaN,AÇUDE CEDRO,25/11/2002,25/11/2012,NaN,NaN
3,CELIBA CIA AGROINDUSTRIAL,04007771-3,METROPOLITANA,IRRIGAÇÃO,1.36,16856.26,NaN,NaN,CANAL DO TRABALHADOR,16/08/2004,16/08/2014,NaN,NaN
4,CELICA CIA AGROINDUSTRIAL,04007770-5,METROPOLITANA,ABASTECIMENTO HUMANO,2.25,13140.00,NaN,NaN,CANAL DO TRABALHADOR,16/08/2004,16/08/2014,NaN,NaN


In [6]:
# Bacias 'criadas' em 2012
new_basins = shapefile_bacias.query("DESCRICA1=='Serra da Ibiapaba' or DESCRICA1=='Sertões de Crateús'")                           
new_basins

,CODIGOBA0,DESCRICA1,IDENTIFI2,AREA3,HECTARES4,REGIONAL5,IMAGENS6,geometry
9,17,Serra da Ibiapaba,11,5721.92,633.89,Crateús,<br>&nbsp;&nbsp;&nbsp;-&nbsp;<a href='http://a...,"POLYGON ((262947.370 9448682.192, 262846.704 9..."
11,24,Sertões de Crateús,12,10793.8,744.41,Crateús,<br>&nbsp;&nbsp;&nbsp;-&nbsp;<a href='http://a...,"MULTIPOLYGON (((262986.026 9448615.007, 262903..."


##### Agora que carregados os pacotes e os dados, vamos ao que interessa...

In [7]:
# Dados de outorga referentes à bacia do Parnaíba
outorga_parnaiba = outorga_2012.query("Bacia=='PARNAÍBA'")
outorga_parnaiba.head()

,Requerente,N Processo,Bacia,Tipo de Uso,Vz Outorgada(L/s),Vol. Outorgado(M3/ano),Latitude,Longitude,Manancial,Data Inicial,Data Final,Unnamed: 11,Unnamed: 12
5,FAZENDA AMWAY NUTRILITE DO BRASIL LTDA,04410334-4,PARNAÍBA,ABASTECIMENTO HUMANO,1.52,34154.51,NaN,NaN,AÇUDE JABURU I,04/03/2005,04/03/2015,NaN,NaN
155,PREFEITURA MUNICIPAL DE VIÇOSA DO CEARÁ,03435760-2,PARNAÍBA,ABASTECIMENTO HUMANO,7.34,95666.50,9604770.0,242377.0,AÇUDE JUA,26/02/2004,26/02/2014,NaN,NaN
211,AGROINDÚSTRIA IBIAPABA LTDA - ME,09698805-3,PARNAÍBA,IRRIGAÇÃO,4.01,64619.19,9597769.0,272139.0,RIACHO TALHADO,03/02/2010,03/02/2014,NaN,NaN
323,DOMINGOS PRADO DAS NEVES,11637979-0,PARNAÍBA,IRRIGAÇÃO,1.96,20826.05,9580398.0,286622.0,POÇO TUBULAR RASO,16/12/2011,16/12/2015,NaN,NaN
331,ASSOCIAÇÃO COMUNITÁRIA DE ITAPERACEMA,12061685-8,PARNAÍBA,ABASTECIMENTO HUMANO,1.04,27338.50,9579540.0,285563.0,BATERIA DE POÇOS,03/05/2012,03/05/2016,NaN,NaN


Veja que as coordenadas não estão no formato mais conhecido, em graus, mas, sim, em metros. Isto ocorre por que existem diferentes sistemas de coordenadas georreferenciadas, os dados de outorga, por exemplo, estão formatados no padrão [SIRGAS 2000](http://www.sirgas.org/pt/sirgas-realizations/sirgas2000/). É **MUITO IMPORTANTE** que quem trabalha com esse tipo de dado preste atenção no **Sistema de Referência de Coordenadas** (SRC) nos projetos, arquivos raster, arquivos vetoriais, banco de dados, entre outros. Assim, Grupo de Pesquisa Petrolífera Européia – *European Petroleum Survey Group* (EPSG) elaborou os chamados "Códigos EPSG", sistematizando todos os SRC do planeta por códigos, com o intuito de organizar essa informação. Você pode conferir os principais Códigos EPSG usados no Brasil [aqui](http://processamentodigital.com.br/2013/07/27/lista-dos-codigos-epsg-mais-utilizados-no-brasil/).

In [8]:
# Array com os pares (Longitude, Lagitude) de cada outorga na bacia do Parnaíba
lon_lat_parnaiba = outorga_parnaiba[["Longitude","Latitude"]].values
lon_lat_parnaiba[:5]

array([[     nan,      nan],
       [ 242377., 9604770.],
       [ 272139., 9597769.],
       [ 286622., 9580398.],
       [ 285563., 9579540.]])

Na Ciência de Dados é importante [*não ficar se repetindo*](https://pt.wikipedia.org/wiki/Don't_repeat_yourself), assim, generalizei um trecho do código passado numa função que a reutilizarei quando for preciso.

In [9]:
#Lista dos dados desagregados
desag_list = get_basin(shapefile_bacias,"DESCRICA1", lon_lat_parnaiba,ifnot_found = "PARNAÍBA_UNDEF",is_upper=True)

100%|██████████| 87/87 [00:05<00:00, 16.28it/s]


In [10]:
# Atualização dos valores das bacias do dataframe outorga_parnaiba
outorga_parnaiba["Bacia"] = desag_list

# Atualização dos valores das bacias do dataframe outorga_2012
outorga_2012.loc[outorga_parnaiba.index,'Bacia'] = desag_list

In [11]:
print("""Das %d outorgas listadas na bacia do Parnaíba em 2012, atualmente fazem parte das bacias:\n
%s
""" % (outorga_parnaiba.shape[0],
       outorga_parnaiba["Bacia"].value_counts()))

Das 87 outorgas listadas na bacia do Parnaíba em 2012, atualmente fazem parte das bacias:

SERRA DA IBIAPABA     64
SERTÕES DE CRATEÚS    17
ACARAÚ                 3
PARNAÍBA_UNDEF         2
COREAÚ                 1
Name: Bacia, dtype: int64



In [15]:
# Agora é só salvar salvar os dados de outorga desagregados
outorga_2012.to_excel(os.path.join(data_path,outorga_data_path,"outorga_2012_desagregado.xlsx"),index=False)